# SARIMA with Cross-Validation

In [ ]:
# import packages

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import gridspec
from matplotlib.patches import Patch
import datetime
import random

In [ ]:
df = pd.read_csv("../scr/data/cleaned_rat_sightings_data/daily_borough_rs.csv")

RMSE=[]

df.head(9)

In [ ]:
cut_off = '2025-06-01'
before_cut_off = '2025-05-31'
last_day = '2025-09-30'

## Simplifications

For now, we focus solely on Manhattan. First, let us recall the baseline model.

## Baseline

In [ ]:
cdate_borough_test = df[df['created_date']>=cut_off]
cdate_borough_test = cdate_borough_test[cdate_borough_test['created_date']<=last_day]

cdate_borough = df[df['created_date']<cut_off]
cdate_borough = cdate_borough[cdate_borough['created_date']>='2020-01-01']


boroughs = [b for b in df['borough'].unique() if pd.notnull(b) and b != 'Unspecified']


def seasonal_average_forecast(data, target_dates, years_back=5, day_window=5):
    df = data.copy()
    df["created_date"] = pd.to_datetime(df["created_date"])
    df["doy"] = df["created_date"].dt.dayofyear
    df["year"] = df["created_date"].dt.year

    forecasts = []
    for target_date in target_dates:
        target_doy = target_date.dayofyear
        target_year = target_date.year
        mask = (
            (df["year"] >= target_year - years_back) &
            (df["year"] < target_year) &
            (np.abs(df["doy"] - target_doy) <= day_window)
        )

        forecasts.append(df.loc[mask, "count"].mean())

    return pd.Series(forecasts, index=target_dates)


# ensure global dataframe is datetime
cdate_borough["created_date"] = pd.to_datetime(cdate_borough["created_date"])


fig = plt.figure(figsize=(50,80))
gs = gridspec.GridSpec(5,1, figure=fig, wspace=0.3, hspace=0.3)

colors = ["r", "b", "g", "purple", "b"]

for i, borough in enumerate(boroughs):
    ax = fig.add_subplot(gs[i])

    borough_data = cdate_borough[cdate_borough["borough"] == borough].assign(created_date=lambda df: pd.to_datetime(df["created_date"])).sort_values("created_date").set_index("created_date")

    # create a complete daily date range
    full_range = pd.date_range(start="2020-01-01", end=before_cut_off, freq="D")

    # reindex and fill missing days with 0
    borough_data = borough_data.reindex(full_range).assign(count=lambda df: df["count"].fillna(0),borough=borough).rename_axis("created_date").reset_index()

    borough_data_test = cdate_borough_test[cdate_borough_test["borough"] == borough].sort_values("created_date").copy()
        
    last_date = cdate_borough["created_date"].max()
    future_dates = pd.date_range(start=last_date + pd.Timedelta(days=1), periods=len(borough_data_test), freq="D")
    
    # compute seasonal-average forecast
    forecast = seasonal_average_forecast(borough_data,future_dates,years_back=5)

    # plot observed data
    ax.plot(borough_data["created_date"].dt.to_pydatetime(), borough_data["count"], "o", color=colors[i], markersize=10, label="Observed")

    # plot forecast
    ax.plot(forecast.index, forecast.values, color="black", linewidth=5, linestyle = "-", label="Seasonal Avg Forecast")

    borough_data_test["created_date"] = pd.to_datetime(borough_data_test["created_date"])
    ax.plot(borough_data_test["created_date"], borough_data_test["count"], "o", color=colors[i], markersize=10, alpha=0.3, label="Observed")

    actual_series = borough_data_test.set_index('created_date')['count']
    actual_aligned = actual_series.reindex(forecast.index, fill_value=0)

    rmse = np.sqrt(np.mean((actual_aligned - forecast.values)**2))
    rss = np.sqrt(np.sum((actual_aligned - forecast.values)**2))

    ax.set_title(f"{borough}", fontsize=35)
    ax.set_xlabel("Date", fontsize=15)
    ax.set_ylabel("Number of Rat Sightings", fontsize=25)
    ax.grid(True)
    ax.set_ylim(0,70)
    ax.tick_params(axis='x', labelsize=24)
    ax.tick_params(axis='y', labelsize=24)
    text_box = Patch(facecolor='white', edgecolor='black', label=f'RMSE: {rmse}')
    text_box2 = Patch(facecolor='white', edgecolor='black', label=f'RSS: {rss}')
    ax.legend(handles=[ax.lines[0], text_box, text_box2], fontsize=22)
    if borough == 'MANHATTAN':
        RMSE.append(rmse)
        
    
    

plt.suptitle("Daily Rat Sightings in NYC: Seasonal Average Forecast", fontsize=36)
plt.show()





## SARIMA on MANHATTAN

We try to model the number of daily rat sightings in Manhattan. We wish to beat the RMSE there.

In [ ]:
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

In [ ]:
rs = df[df['borough']=='MANHATTAN']

rs['created_date'] = pd.to_datetime(rs['created_date']) 

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(55,36))
sm.graphics.tsa.plot_acf(rs['count'], lags = 365*2, ax=ax)
plt.xlabel("Lag",fontsize=24)
plt.ylabel("Autocorrelation",fontsize=30)
plt.xticks(fontsize=24)
plt.yticks(fontsize=24)
plt.show()

fig, ax = plt.subplots(1, 1, figsize=(55,36))
sm.graphics.tsa.plot_pacf(rs['count'], lags = 365*2, ax=ax)
plt.xlabel("Lag",fontsize=24)
plt.ylabel("Partial Autocorrelation",fontsize=30)
plt.xticks(fontsize=24)
plt.yticks(fontsize=24)
plt.show()

In [ ]:
import statsmodels.tsa.api as sm
from pmdarima import auto_arima

In [ ]:
y = rs
plt.figure(figsize=(30,5))
plt.plot(y['created_date'], y['count'])
plt.show()

In [ ]:
y_train = y[y['created_date']< cut_off]
y_test = y[y['created_date']>= cut_off]

In [ ]:
# A SARIMA model is not wise for seasonality of 1 year with daily data. Picking m = 7 is NOT appropriate here unless we 
# see seasonality with 7 day periods.
# See https://alkaline-ml.com/pmdarima/2.0.1/tips_and_tricks.html?highlight=seasonal


# Uncomment the line below to find AIC minimizing values to use for the ARIMA model. 
z = y_train['count'].to_numpy()
#auto_arima(z, trace=True, error_action="ignore", stepwise=True, seasonal=True, m = 3675)

In [ ]:
model = sm.ARIMA(z, order = (2, 1, 4), seasonal_order=(0,1,2,365)).fit()
print(model.summary())
plt.figure(figsize=(40,10))
plt.plot(y_train['created_date'], y_train['count'], label="Training Data")
plt.plot(y_test['created_date'], y_test['count'], label="Test Data")

plt.plot(y_train['created_date'], model.fittedvalues, label="Fitted SARIMA Model")
plt.plot(y_test['created_date'], model.forecast(len(y_test['created_date'])), label="SARIMA Forecast")


rmse = np.sqrt(np.mean((y_test['count'] - model.forecast(len(y_test['created_date'])))**2))

RMSE.append(rmse)

rss = np.sqrt(np.sum((y_test['count'] - model.forecast(len(y_test['created_date'])))**2))
text_box = Patch(facecolor='white', edgecolor='black', label=f'RMSE: {rmse:.2f}')
text_box2 = Patch(facecolor='white', edgecolor='black', label=f'RSS: {rss:.2f}')

handles, labels = plt.gca().get_legend_handles_labels()
handles.extend([text_box, text_box2])
labels.extend([f"RMSE: {rmse:.6f}", f"RSS: {rss:.6f}"])

plt.legend(handles=handles, labels=labels, fontsize=18)
plt.show()

In [ ]:
print(f"For Manhattan and forecast from {cut_off} until {last_day}.")
print(f"Improved from RMSE of {float(RMSE[0])} to {float(RMSE[1])}.\n")
print(f"This is a {round((1-(RMSE[1]/RMSE[0]))*100, 4)}% reduction.")

## SARIMA with Cross-Validation Test

In [ ]:
df['created_date'] = pd.to_datetime(df['created_date'])

Xnew =  df.loc[df['borough'] == 'MANHATTAN'].set_index('created_date').sort_index()['count'].asfreq('D')
Xnew = Xnew.fillna(0)

In [ ]:
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Initialize lists to store metrics for each split
mae_list = []
mse_list = []
rmse_list = []
mape_list = []
# Parameters for rolling window
window_size = 365  # Size of the initial training window
test_size = 365    # Size of the test set for each rolling step
X = Xnew  # Assuming this is the time series data
X.fillna(0)
# Loop through the data with a rolling window approach
for start in range(0, len(X) - window_size - test_size + 1, 7):
    # Define the training and test sets
    train = X[start:start + window_size]
    test = X[start + window_size:start + window_size + test_size]
    # Fit the SARIMA model on the training data
    model = SARIMAX(train, order=(1, 1, 1), seasonal_order=(0, 1, 2, 7),
                    enforce_stationarity=False, enforce_invertibility=False)

    # it is unclear if these the best orders to use.


    results = model.fit(disp=False, maxiter=50)
    # Forecast on the test data
    forecasts = results.predict(start=len(train), end=len(train) + len(test) - 1)
    # Calculate evaluation metrics
    mae = mean_absolute_error(test, forecasts)
    mse = mean_squared_error(test, forecasts)
    rmse = np.sqrt(mse)
    mape = np.mean(np.abs((test - forecasts) / test)) * 100
    # Append metrics to their respective lists
    mae_list.append(mae)
    mse_list.append(mse)
    rmse_list.append(rmse)
    mape_list.append(mape)
    # Print evaluation metrics for this split
    print(f'Rolling Window {start + 1} - {start + window_size} | Test Period: {start + window_size + 1} - {start + window_size + test_size}')
    print(f'MAE for this split: {mae}')
    print(f'MSE for this split: {mse}')
    print(f'RMSE for this split: {rmse}')
    print(f'MAPE for this split: {mape:.2f}%\n')
# Calculate and print average values across all splits
average_mae = np.mean(mae_list)
average_mse = np.mean(mse_list)
average_rmse = np.mean(rmse_list)
average_mape = np.mean(mape_list)
print(f'\nAverage MAE: {average_mae}')
print(f'Average MSE: {average_mse}')
print(f'Average RMSE: {average_rmse}')
print(f'Average MAPE: {average_mape:.2f}%')

In [ ]:
mae_list

In [ ]:
mse_list


In [ ]:
rmse_list

In [ ]:
mape_list